### 1.1 Update score preprocessing script

This is how we can update score preprocessing script

In [2]:
%%writefile ../score_preprocessing/score_processing_script.py
import pandas as pd
import numpy as np
import boto3
def main():
    data = pd.read_csv('s3://poc-sagemaker-step-function-bucketformodelanddata-iip9wy7qh0y0/dataset/bank-additional-full.csv', sep=';')
    # Preprocess the data set
    data['no_previous_contact'] = np.where(data['pdays'] == 999, 1, 0)                                 # Indicator variable to capture when pdays takes a value of 999
    data['not_working'] = np.where(np.in1d(data['job'], ['student', 'retired', 'unemployed']), 1, 0)   # Indicator for individuals not actively employed
    model_data = pd.get_dummies(data)
    scoring_data = model_data.drop(['duration', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'], axis=1)    
    scoring_data.drop(['y_no', 'y_yes'], axis=1).to_csv('scoring.csv', index=False, header=False)
    bucket = 'poc-sagemaker-step-function-bucketformodelanddata-iip9wy7qh0y0'
    boto3.Session().resource('s3').Bucket(bucket).Object( 'scoring/scoring.csv').upload_file('scoring.csv')
    print('Finished running processing job')
if __name__ == "__main__":
    main()


Overwriting ../score_preprocessing/score_processing_script.py


### 1.2 Update score preprocessing parameter

This is how we can update score parameters

In [3]:
%%writefile ../score_hyperparameters/parameter.json
{
    "InputS3Location": "",
    "OutputS3Location":""
}

Overwriting ../score_hyperparameters/parameter.json


### 1.3 Trigger scoring pipeline 

In [4]:
! aws s3 cp ../score_hyperparameters/parameter.json s3://poc-sagemaker-step-function-bucketformodelanddata-iip9wy7qh0y0/scoring/trigger_scoring/

upload: ../score_hyperparameters/parameter.json to s3://poc-sagemaker-step-function-bucketformodelanddata-iip9wy7qh0y0/scoring/trigger_scoring/parameter.json


In [3]:
!git pull origin dev

remote: Counting objects: 34, done.
Unpacking objects: 100% (34/34), done.
From ssh://git-codecommit.us-east-1.amazonaws.com/v1/repos/poc-sagemaker-step-function
 * branch            dev        -> FETCH_HEAD
   ab6ee0e..c66cd91  dev        -> origin/dev
Updating ab6ee0e..c66cd91
Fast-forward
 buildspec.yml                        | 8 +++++---
 cloudformation/scoring_cfn.yaml      | 2 +-
 cloudformation/training_cfn.yml      | 8 ++++++--
 train_hyperparameters/parameter.json | 2 +-
 4 files changed, 13 insertions(+), 7 deletions(-)
